In [1]:
import os
import ast
import json
import glob
import torch
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import gc
#import cudf
import joblib

from tqdm import tqdm
from datetime import datetime
from collections import Counter
from sklearn import preprocessing

In [2]:
train_w1 = dict()
label_w1 = dict()
# train_w2 = dict()
# train_w3 = dict()
# train_w4 = dict()

for file in glob.glob("/kaggle/input/otto-prep-4-weeks/train_w0_part*.parquet"):
   train_w1[file] = pd.read_parquet(file)
train_w1 = pd.concat(train_w1.values())

for file in glob.glob("/kaggle/input/otto-prep-4-weeks/label_w0_part*.parquet"):
   label_w1[file] = pd.read_parquet(file)
label_w1 = pd.concat(label_w1.values())

# for file in glob.glob("/kaggle/input/otto-training-wo-split/train_w1_part*.parquet"):
#    train_w2[file] = pd.read_parquet(file)
# train_w2 = pd.concat(train_w2.values())

# for file in glob.glob("/kaggle/input/otto-training-wo-split/train_w2_part*.parquet"):
#    train_w3[file] = pd.read_parquet(file)
# train_w3 = pd.concat(train_w3.values())

# for file in glob.glob("/kaggle/input/otto-training-wo-split/train_w3_part*.parquet"):
#    train_w4[file] = pd.read_parquet(file)
# train_w4 = pd.concat(train_w4.values())


#max aid unique 486

In [3]:
#label encoding aids

def get_le():
    temp_all_data = pd.read_parquet('/kaggle/input/otto-full-optimized-memory-footprint/train.parquet')


    le = preprocessing.LabelEncoder()
    le.fit(temp_all_data['aid'])
    del temp_all_data
    return le


def label_encoding(df,le):


    df['aid'] = le.transform(df['aid'])

    #set start = 2 to reserve 0 for mask and 1 for pad
    df['aid+2'] = df['aid']+2
    df['aid'] = df['aid+2']

    df = df.drop(['aid+2'], axis = 1)
    joblib.dump(le, 'label_encoder.joblib')
    return df

In [4]:
le = get_le()
train_w1 = label_encoding(train_w1, le)

In [5]:
def get_merged(df, label_df):

    df = pd.DataFrame(df.groupby('session')['aid'].unique().agg(list))
    label_df = pd.DataFrame(label_df.groupby('session')['aid'].unique().agg(list))

    label_df.rename(columns = {'aid': 'label'}, inplace = True)

    df = df.reset_index()
    label_df = label_df.reset_index()

    df = pd.merge(df, label_df, on = "session", how = "inner")
    return df

In [6]:
train_w1 = get_merged(train_w1, label_w1)
del label_w1

In [7]:
TRAIN_HISTORY = 486
PREDICTION_HISTORY = 433

In [8]:
def get_input_target(df, CHUNK = 10000):
    train_w_dict = {}

    it = len(train_w1)//CHUNK +1

    for i in range(it):
        if i < it-1: temp = df.iloc[i*CHUNK: (i+1)*CHUNK,:].copy()
        else: temp = df.iloc[i*CHUNK:,:].copy()
        temp['input'] = temp.apply(lambda x: x['aid'] + 
                                      list((TRAIN_HISTORY - len(x['aid'])) * [0]), 
                                      axis = 1)
        temp['target'] = temp.apply(lambda x: x['label'] + 
                                      list((PREDICTION_HISTORY - len(x['aid'])) * [0]), 
                                      axis = 1)

        train_w_dict[i] = temp
        del temp

    df =  pd.concat(train_w_dict.values())
    return df
    

In [ ]:
train_w1 = get_input_target(train_w1)

In [ ]:
# class Dataset(torch.utils.data.Dataset):
#     def __init__(self, df, history_size=120):
#         self.data = df
#         self.history_size = history_size

#     def __len__(self):
#         return len(self.data)
    
#     def __getitem__(self, idx):
        